In [74]:
from keras import backend as K
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Input, Dense, Flatten, Activation
from keras.layers import Reshape, Lambda
from keras.layers.merge import add, concatenate
from keras.models import Model
from keras.optimizers import rmsprop
def create_model(input_shape):
    # Network parameters
    act = 'relu'

    input_data = Input(name='the_input', shape=input_shape, dtype='float32')

    inner = Conv2D(6, (5,5), padding='valid',
                   activation=act, kernel_initializer='he_normal',
                   name='conv1')(input_data)
    inner = MaxPooling2D(pool_size=(2, 2), name='max1')(inner)
    inner = Conv2D(16, (5,5), padding='valid',
                   activation=act, kernel_initializer='he_normal',
                   name='conv2')(inner)
    inner = MaxPooling2D(pool_size=(2, 2), name='max2')(inner)
    
    inner = Conv2D(16, (5,5), padding='valid',
                   activation=act, kernel_initializer='he_normal',
                   name='conv3')(inner)
    
    inner = Flatten(name='flatten1')(inner)
    
    inner = Dense(120, activation=act, name='dense1')(inner)
    
    inner = Dense(84, activation=act, name='dense2')(inner)
    
    y_pred = Activation('softmax', name='softmax')(inner)
    
    Model(inputs=input_data, outputs=y_pred).summary()



In [28]:
LeNetModel = create_model((32,32,1))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       (None, 32, 32, 1)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 28, 28, 6)         156       
_________________________________________________________________
max1 (MaxPooling2D)          (None, 14, 14, 6)         0         
_________________________________________________________________
conv2 (Conv2D)               (None, 10, 10, 16)        2416      
_________________________________________________________________
max2 (MaxPooling2D)          (None, 5, 5, 16)          0         
_________________________________________________________________
conv3 (Conv2D)               (None, 1, 1, 16)          6416      
_________________________________________________________________
flatten1 (Flatten)           (None, 16)                0         
__________

In [30]:
import os 
def ReadAllData(rootDir): 
    
    all_data = []
    for class_idx in range(8):
        data_path  = rootDir + "/" + str(class_idx+1) + "/"
        for lists in os.listdir(data_path): 
            path = os.path.join(data_path, lists) 
            if os.path.isdir(path): 
                print("This is dir")
                continue
            one_data = {}
            one_data["file_name"] = path
            one_data["label"] = class_idx + 1
            all_data.append(one_data)
    return all_data

In [31]:
all_data = ReadAllData("E:/Deeplearning/AICar/classify/classify")
print(all_data)

[{'file_name': 'E:/Deeplearning/AICar/classify/classify/1/20180908-074211-893000-orisign-1.jpg', 'label': 1}, {'file_name': 'E:/Deeplearning/AICar/classify/classify/1/20180908-074517-615000-orisign-1.jpg', 'label': 1}, {'file_name': 'E:/Deeplearning/AICar/classify/classify/1/20180908-074805-661000-orisign-1.jpg', 'label': 1}, {'file_name': 'E:/Deeplearning/AICar/classify/classify/1/20180908-075214-549000-orisign-1.jpg', 'label': 1}, {'file_name': 'E:/Deeplearning/AICar/classify/classify/1/20180908-075407-570000-orisign-1.jpg', 'label': 1}, {'file_name': 'E:/Deeplearning/AICar/classify/classify/1/20180908-075648-234000-orisign-1.jpg', 'label': 1}, {'file_name': 'E:/Deeplearning/AICar/classify/classify/1/20180908-075942-483000-orisign-1.jpg', 'label': 1}, {'file_name': 'E:/Deeplearning/AICar/classify/classify/1/20180908-080352-200000-orisign-1.jpg', 'label': 1}, {'file_name': 'E:/Deeplearning/AICar/classify/classify/1/20180908-080638-930000-orisign-1.jpg', 'label': 1}, {'file_name': 'E:/

In [33]:
import numpy as np
val_split_ratio = 0.9 
np.random.shuffle(all_data)
val_split = int(val_split_ratio * len(all_data))
train_data_list = all_data[0:val_split]
val_data_list = all_data[val_split:]

In [119]:
import cv2
def get_batch(index, size, train_type):
    x_data = []
    
    y_data= []
    if train_type == 0: 
        #Read train data
        np.random.shuffle(train_data_list)
        for idx in range(size):
            image_idx = (index+idx)%len(train_data_list)
            image_name = train_data_list[image_idx]["file_name"]
            image_label = train_data_list[image_idx]["label"]
            src_image = cv2.imread(image_name)
            gray_image = cv2.cvtColor(src_image,cv2.COLOR_BGR2GRAY)
            gray_image = cv2.resize(gray_image,(32,32),interpolation=cv2.INTER_CUBIC)
            input_data = gray_image[:,:,np.newaxis]
            
            image_label_one_hot = [0]*8
            image_label_one_hot[image_label-1] = 1
            x_data.append(input_data)
            y_data.append(image_label_one_hot)
    else:
        #Reade val data
        np.random.shuffle(val_data_list)
        for idx in range(size):
            image_idx = (index+idx)%len(val_data_list)
            image_name = val_data_list[image_idx]["file_name"]
            image_label = val_data_list[image_idx]["label"]
            src_image = cv2.imread(image_name)
            gray_image = cv2.cvtColor(src_image,cv2.COLOR_BGR2GRAY)
            gray_image = cv2.resize(gray_image,(32,32),interpolation=cv2.INTER_CUBIC)
            input_data = gray_image[:,:,np.newaxis]
            
            image_label_one_hot = [0]*8
            image_label_one_hot[image_label-1] = 1
            x_data.append(input_data)
            y_data.append(image_label_one_hot)
    
    return (np.array(x_data), np.array(y_data))
test_data = get_batch(0,2,0)
print(test_data[0].shape)


(2, 32, 32, 1)


In [120]:
cur_train_index = 0
minibatch_size = 16
def next_train():
    global cur_train_index
    while 1:
        ret = get_batch(cur_train_index, minibatch_size, train_type=0)
        cur_train_index += minibatch_size
        if cur_train_index >= len(train_data_list):
            cur_train_index = cur_train_index%minibatch_size
        yield ret

In [121]:
cur_val_index = 0
def next_val():
    global cur_val_index
    while 1:
        ret = get_batch(cur_val_index, minibatch_size, train_type=1)
        cur_val_index += minibatch_size
        if cur_val_index >= len(val_data_list):
            cur_val_index = cur_val_index%minibatch_size
        yield ret

In [124]:
from keras.optimizers import SGD
def train_LeNet(input_shape):
    # Network parameters
    act = 'relu'

    input_data = Input(name='the_input', shape=input_shape, dtype='float32')

    inner = Conv2D(32, (5,5), padding='valid',
                   activation=act, kernel_initializer='he_normal',
                   name='conv1')(input_data)
    inner = MaxPooling2D(pool_size=(2, 2), name='max1')(inner)
    inner = Conv2D(32, (5,5), padding='valid',
                   activation=act, kernel_initializer='he_normal',
                   name='conv2')(inner)
    inner = MaxPooling2D(pool_size=(2, 2), name='max2')(inner)
    
    inner = Conv2D(32, (5,5), padding='valid',
                   activation=act, kernel_initializer='he_normal',
                   name='conv3')(inner)
    
    inner = Flatten(name='flatten1')(inner)
    
    inner = Dense(120, activation=act, name='dense1')(inner)
    
    inner = Dense(84, activation=act, name='dense2')(inner)
    
    inner = Dense(8, activation=act, name='dense3')(inner)
    
    y_pred = Activation('softmax', name='softmax')(inner)
    
    model = Model(inputs=input_data, outputs=y_pred, name='lenet_5')
    
    model.summary()
    
    return model
my_LeNet = train_LeNet((32,32,1))    
minibatch_size = 16
my_LeNet.compile(
    loss='categorical_crossentropy',
    #optimizer=SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5),
    optimizer = rmsprop(lr=0.0001),
    metrics=['accuracy']
    )

my_LeNet.fit_generator(generator=next_train(),
                        steps_per_epoch=400,  #400
                        epochs=5,
                        validation_data=next_val(),
                        validation_steps=50)
    


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       (None, 32, 32, 1)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 28, 28, 32)        832       
_________________________________________________________________
max1 (MaxPooling2D)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 10, 10, 32)        25632     
_________________________________________________________________
max2 (MaxPooling2D)          (None, 5, 5, 32)          0         
_________________________________________________________________
conv3 (Conv2D)               (None, 1, 1, 32)          25632     
_________________________________________________________________
flatten1 (Flatten)           (None, 32)                0         
__________

In [131]:
import time
test_data = get_batch(0,1,0)
real_resul = test_data[1]
start_time = time.time()
predict_result = my_LeNet.predict(test_data[0])
end_time = time.time()
print("real_resul", np.argmax(real_resul))
print("predict_result", np.argmax(predict_result))
print("cost time",end_time - start_time)

real_resul 3
predict_result 3
cost time 4.708551645278931


In [133]:
test_data = get_batch(0,1,0)
real_resul = test_data[1]
start_time = time.time()
predict_result = my_LeNet.predict(test_data[0])
end_time = time.time()
print("real_resul", np.argmax(real_resul))
print("predict_result", np.argmax(predict_result))
print("cost time",end_time - start_time)

real_resul 3
predict_result 3
cost time 0.0050122737884521484


In [134]:
my_LeNet.save("myLeNet.h5")

In [149]:
from keras.models import load_model
load_LeNet = load_model('myLeNet.h5')

In [152]:
test_data = get_batch(0,1,0)
real_resul = test_data[1]
start_time = time.time()
predict_result = load_LeNet.predict(test_data[0])
end_time = time.time()
print("real_resul", np.argmax(real_resul))
print("predict_result", np.argmax(predict_result))
print("cost time",end_time - start_time)

real_resul 1
predict_result 1
cost time 0.005012989044189453


In [138]:
start_time = time.time()
predict_result = load_LeNet.predict(test_data[0])
end_time = time.time()
print("real_resul", np.argmax(real_resul))
print("predict_result", np.argmax(predict_result))
print("cost time",end_time - start_time)

real_resul 2
predict_result 2
cost time 0.005013704299926758


In [130]:
# minibatch_size = 16
# my_LeNet.compile(
#     loss='categorical_crossentropy',
#     #optimizer=SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5),
#     optimizer = rmsprop(lr=0.0001),
#     metrics=['accuracy']
#     )

# my_LeNet.fit_generator(generator=next_train(),
#                         steps_per_epoch=400,  #400
#                         epochs=5,
#                         validation_data=next_val(),
#                         validation_steps=50)

Epoch 1/5


KeyboardInterrupt: 

In [118]:
my_LeNet.compile(
    loss='categorical_crossentropy',
    optimizer=SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5),
    metrics=['accuracy']
    )

my_LeNet.fit_generator(generator=next_train(),
                        steps_per_epoch=400,  #400
                        epochs=5,
                        validation_data=next_val(),
                        validation_steps=50)

Epoch 1/5
273/400 [===================>..........] - ETA: 7s - loss: 2.0794 - acc: 0.1445

KeyboardInterrupt: 

In [93]:
import time
test_data = get_batch(0,1,0)
real_resul = test_data[1]
start_time = time.time()
predict_result = my_LeNet.predict(test_data[0])
end_time = time.time()
print("real_resul", np.argmax(real_resul))
print("predict_result", np.argmax(predict_result))
print("cost time",end_time - start_time)

real_resul 2
predict_result 2
cost time 0.0050127506256103516
